In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "loGu88Z",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "u6p1OlcKS8K0ulvvGBdVKw",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
import os
print "current dir: ", os.getcwd()
os.chdir("/home/foodmap/food101/")
print "current dir: ", os.getcwd()

current dir:  /home/foodmap/food101/notebooks
current dir:  /home/foodmap/food101


In [3]:
from stream2index import streamTweets
from stream2index.Stream2Index import Stream2Index
from elasticsearch import Elasticsearch
import time
from twython import TwythonRateLimitError

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [ ]:
#### PUT CONFIG HERE
CONSUMER_KEY = 'JzAQ0Y2GbBg900GENP444u24K'
CONSUMER_SECRET = '5Mu6TnLR2oC7G9Z87egRzJU2X8vV3mL17qI5jBkL75g3wuLi5c'
ACCESS_TOKEN = '308456550-i6JR7eJo8H0VfV1X0Ujg06YhL9rJOFD3Pgey3zbi'
ACCESS_TOKEN_SECRET = 'J2rY8MCjkDblQpI3LPCXtG7v6nNO7K3YOOZbTASUGiRni'

# Minimal time accepted between two Rate Limit Errors
TOO_SOON = 10
# Time to wait if we receive a Rate Limit Error too soon after a previous one
WAIT_SOME_MORE = 60

In [ ]:
htString = streamTweets.getKeywordsList()

In [ ]:
stream = Stream2Index(CONSUMER_KEY, CONSUMER_SECRET,
                        ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
try:
    # FILTER
    stream.statuses.filter(track=htString)
except TwythonRateLimitError as e:
    # If this error is received after only few calls (10 seconds of calls) wait just a minute
    if time.time() - start < TOO_SOON:
        log('Waiting %s seconds more for resuming download after recurrent rate limit error ...'
            % WAIT_SOME_MORE)
        time.sleep(WAIT_SOME_MORE)
    else:
        log(e, id)
        log('Waiting %s seconds for resuming download after rate limit error ...')
        time.sleep(60)

In [4]:
es = Elasticsearch(['localhost'], http_auth=('elastic', 'changeme'), port=9200)

In [ ]:
es.get(index='stream', doc_type='tweet_snippet', id=876794234182844418)

In [ ]:
es.count(index='stream', doc_type='tweet_snippet')

In [7]:
es.delete_by_query(index='stream', doc_type='tweet_snippet', body={
    'query': {
        "match_all" : {}
    }
})

2017-06-20 18:18:10,228 POST http://localhost:9200/stream/tweet_snippet/_delete_by_query [status:200 request:0.058s]


{u'batches': 1,
 u'deleted': 49,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 30,
 u'total': 49,
 u'version_conflicts': 0}

In [ ]:
import json
from index.preprocess_tweet import get_image_category
results = es.search(index='stream', doc_type='tweet_snippet', size=100, body={
  'query': {
    'match': {
      'img_category': 'Sushi',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

# print results
# print len(results["hits"]["hits"])


request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)
    print tweet["_source"]["text"]
    print img_url
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

2017-06-20 17:52:59,886 GET http://localhost:9200/stream/tweet_snippet/_search?size=100 [status:200 request:0.014s]


Mon #lunch @bonhomieparis in the 10th. Two thumbs up. #paris https://t.co/qyK5e3AWUX
http://pbs.twimg.com/media/DCr_gc1XgAAGbIE.jpg
[]
None


Chocolate coating with nuts? How can you say no? #SwensensIceCream #Swensens #icecream https://t.co/WDi9kqfryC
http://pbs.twimg.com/media/DCr_2OKXkAAY1b1.jpg
[{u'score': 10.0, u'label': u'ice_cream'}]
ice_cream


#vegas #buffet #restaurant #2for1 #menu #reservation #openbar #brunch #dinner #lunch https://t.co/Oom3EQTIP8
http://pbs.twimg.com/media/DCr_s4cXUAEyu-K.jpg
[]
None


Nothing Beats a Hot FRESH PIZZA !!#pizza #OrlandPark #Italianvillapizza https://t.co/jiFrAl9qDA
http://pbs.twimg.com/media/DCr_weWXUAEKFIv.jpg
[]
None


Thinking of having a Calzone? Call us @ 708-403-3335 #pizza #italianvilllapizza #OrlandPark #calzones https://t.co/PB1yZpQV0J
http://pbs.twimg.com/media/DCr_xVCWAAAHnOg.jpg
[{u'score': 5.0, u'label': u'omelette'}, {u'score': 5.0, u'label': u'chicken_quesadilla'}]
omelette


Will Amazon change the food business &amp;mdash; or

### get all docs from index

In [ ]:
res = es.search(index="stream", doc_type='tweet_snippet', size=100, body={"query": {"match_all": {}}})
[x['_source'] for x in res['hits']['hits']]

In [ ]:
import requests
import json

response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
json_data = json.loads(response.text)
json_data